In [ ]:
import requests
import json
import pandas as pd
import io

In [ ]:
request_url = 'https://www.regionalstatistik.de/genesis/online'
request_params = {
    "operation": "download",
    "code": "31231-01-02-4",
    "startjahr":"2000",
    "endjahr": "2005",
    "zeiten": "",
    "option":"csv", #(csv, ffcsv, xlsx, xml)
    "zeitscheiben": "",  #Anzahl der aktuellsten Zeitscheiben, die tabelliert werden sollen
    "language": "de"
}
# save the response
de_response = requests.get(url=request_url, params=request_params)
de_response

In [ ]:
with open('data/year_2000_2005.csv', 'wb') as f:
    f.write(de_response.content)
data = pd.read_csv('data/year_2000_2005.csv',delimiter=';', skiprows=4, skipfooter=4, header=[0,1,2], encoding = 'latin1', decimal=',')
data.columns = data.columns.map('_'.join)
data.columns = data.columns.str.replace(' ','_')
data.drop(columns=[
       'Gebäude_nach_Anzahl_der_Wohnungen_1_Wohnung_Anzahl',
       'Gebäude_nach_Anzahl_der_Wohnungen_2_Wohnungen_Anzahl',
       'Größe_der_Wohnung_Wohnungen_mit_1_Raum_Anzahl',
       'Größe_der_Wohnung_Wohnungen_mit_2_Räumen_Anzahl',
       'Größe_der_Wohnung_Wohnungen_mit_3_Räumen_Anzahl',
       'Größe_der_Wohnung_Wohnungen_mit_4_Räumen_Anzahl',
       'Größe_der_Wohnung_Wohnungen_mit_5_Räumen_Anzahl',
       'Größe_der_Wohnung_Wohnungen_mit_6_Räumen_Anzahl',
       'Größe_der_Wohnung_Wohnungen_mit_7_Räumen_oder_mehr_Anzahl',
       'Unnamed:_15_level_0_Unnamed:_15_level_1_Anzahl'], inplace=True)
data.rename(columns={'Unnamed:_0_level_0_Unnamed:_0_level_1_Unnamed:_0_level_2': 'year',
       'Unnamed:_1_level_0_Unnamed:_1_level_1_Unnamed:_1_level_2':'code',
       'Unnamed:_2_level_0_Unnamed:_2_level_1_Unnamed:_2_level_2':'district',
       'Gebäude_nach_Anzahl_der_Wohnungen_Insgesamt_Anzahl':'number_of_buildings',
       'Unnamed:_6_level_0_Unnamed:_6_level_1_1000_qm': 'living_space_1000qm',
       'Größe_der_Wohnung_Insgesamt_Anzahl' : 'number_of_apartments'
},inplace=True)
data["year"]= pd.to_datetime(data["year"]).dt.year
data["number_of_buildings"]= pd.to_numeric(data["number_of_buildings"],errors='coerce')
data["number_of_apartments"]= pd.to_numeric(data["number_of_apartments"],errors='coerce')
data["living_space_1000qm"]= pd.to_numeric(data["living_space_1000qm"],errors='coerce')
data = data.query('code.str.len() == 5')
data['country'] = ['Germany']*data.shape[0]
data['state_code'] = data['code'].map(lambda x: int(str(x)[:2]))
state_maping = {
    1 : "Schleswig-Holstein",
    2 : "Hamburg",
    3 : "Niedersachsen",
    4 : "Bremen",
    5 : "Nordrhein-Westfalen",
    6 : "Hessen",
    7 : "Rheinland-Pfalz",
    8 : "Baden-Württemberg",
    9 : "Bayern",
    10 : "Saarland",
    11 : "Berlin",
    12 : "Brandenburg",
    13 : "Mecklenburg-Vorpommern",
    14 : "Sachsen",
    15 : "Sachsen-Anhalt",
    16 : "Thüringen",
    }
data['state'] = data['state_code'].map(state_maping)

In [ ]:
#data.query('number_of_buildings != number_of_buildings')
#data['number_of_apartments'] = pd.to_numeric(data['number_of_apartments'], errors='coerce')
#data['number_of_buildings'] = pd.to_numeric(data['number_of_buildings'], errors='coerce')
#data.query('number_of_buildings != number_of_buildings')
#data.query('code 032')
#data.drop(columns='new_code',inplace=True)

In [ ]:
data_2000_2010 = pd.concat([data_2006_2010,data])